In [ ]:
import random
import numpy as np
from scipy.stats import ttest_ind_from_stats

### Задание 1
Подсчитать количество клиентов для каждого из сегментов, совершавших транзакции. 

In [ ]:
def unique_segment_clients(segment):
    unique_users = set()
    with open('transactions.txt', 'r') as file:
        for transaction in file:
            _, user_id, _, category = transaction.strip().split(',')
            if category == segment:
                unique_users.add(user_id)
    return len(unique_users)

In [ ]:
print("Число клиентов, совершавших транзакции в секторе R:", unique_segment_clients('R'))
print("Число клиентов, совершавших транзакции в секторе AF:", unique_segment_clients('AF'))

### Задание 2
Подсчитать средний объем отдельной транзакции в каждом из сегментов.

In [ ]:
def segment_mean(segment):
    segment_volume = 0
    segment_transactions_amount = 0

    with open('transactions.txt', 'r') as file:
        for transaction in file:
            _, _, volume, category = transaction.strip().split(',')
            if category == segment:
                segment_transactions_amount += 1
                segment_volume += int(volume)
    return segment_volume / segment_transactions_amount    

In [ ]:
print('Средний объем отдельной транзакции в сегменте R: ', segment_mean('R'))
print('Средний объем отдельной транзакции в сегменте AF: ', segment_mean('AF'))   

### Задание 3
Подсчитать 90% доверительный интервал для среднего объема отдельной транзакции в каждом из сегментов. 

In [ ]:
# Получение выборки из генеральной совокупности для сегментов
def generate_sample(segment, sample_size=5000):
    sample = []

    with open('transactions.txt', 'r') as file:
        for transaction in file:
            _, _, volume, category = transaction.strip().split(',')
            if category == segment:
                sample.append(int(volume))
            if len(sample) >= sample_size:
                break
    return sample

def confidence_interval(segment, sample_size=5000):    
    z = 1.65  # Значение для 90% доверительного интервала
    sample = generate_sample(segment, sample_size)  # Выборка из сегмента
    sample_std = np.std(sample)
    segment_mean_value = round(segment_mean(segment))
    left_border = segment_mean_value - z * sample_std / np.sqrt(sample_size)
    right_border = segment_mean_value + z * sample_std / np.sqrt(sample_size)
    return left_border, right_border

In [ ]:
R_left_border, R_right_border = confidence_interval('R', sample_size=1000)
print("90% Доверительный интервал для сегмента R: ", str(R_left_border) + " <= mu <= " + str(R_right_border)) 
AF_left_border, AF_right_border = confidence_interval('AF', sample_size=1000)
print("90% Доверительный интервал для сегмента AF: ", str(AF_left_border) + " <= mu <= " + str(AF_right_border)) 

### Задание 4
Проверить гипотезу о равенстве средних объемов отдельных транзакций между сегментами при уровне значимости 10%. 

In [ ]:
def huge_data_statistics(segment):
    common_sum = 0
    observations_number = 0
    segment_mean_value = round(segment_mean(segment))
    with open('transactions.txt', 'r') as file:
        for transaction in file:
            _, _, volume, category = transaction.strip().split(',')
            if category == segment:
                common_sum += (int(volume) - segment_mean_value)**2
                observations_number += 1
    return segment_mean_value, np.sqrt(common_sum / observations_number), observations_number

In [ ]:
R_mean, R_std, R_observations_number = huge_data_statistics('R')
AF_mean, AF_std, AF_observations_number = huge_data_statistics('AF')
test_results = ttest_ind_from_stats(R_mean, R_std, R_observations_number, AF_mean, AF_std, AF_observations_number)
if test_results.pvalue > 0.1:
    print("Невозможно отклонить нулевую гипотезу о равенстве средних объемов отдельных транзакций между сегментами\
 при уровне значимости 10%")
else:
    print("Возможно отклонить нулевую гипотезу о равенстве средних объемов отдельных транзакций между сегментами\
 при уровне значимости 10%")